In [14]:
from pathlib import Path


import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib
import torchvision
from torchvision import datasets, transforms, utils
import torchvision.models as models
from tqdm import tqdm
from PIL import Image

import numpy as np
from torch.optim import lr_scheduler


from pycocotools.coco import COCO
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)# Import Libraries

# For visualization
import seaborn as sns
from matplotlib import colors
from tensorboard.backend.event_processing import event_accumulator as ea

# Scipy for calculating distance
from scipy.spatial import distance

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import os, json, cv2, random, time, copy

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device 객체
# Set base params
plt.rcParams["figure.figsize"] = [16,9]

In [24]:
register_coco_instances("car_data", {}, './inference/annotations.json', './inference/test_img')

In [25]:

dataset= DatasetCatalog.get("car_data")
dataset

[08/25 01:29:43 d2.data.datasets.coco]: Loaded 461 images in COCO format from ./inference/annotations.json


[{'file_name': './inference/test_img/1.jpg',
  'height': 1024,
  'width': 1024,
  'image_id': 1,
  'annotations': [{'iscrowd': 0,
    'bbox': [370, 147, 182, 481],
    'category_id': 0,
    'segmentation': [[469,
      150,
      451,
      156,
      433,
      183,
      408,
      254,
      395,
      295,
      377,
      380,
      370,
      439,
      370,
      491,
      399,
      552,
      414,
      583,
      439,
      604,
      460,
      623,
      494,
      628,
      521,
      600,
      528,
      554,
      543,
      493,
      549,
      450,
      552,
      399,
      549,
      331,
      549,
      267,
      543,
      202,
      519,
      165,
      496,
      151,
      473,
      147]],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'iscrowd': 0,
    'bbox': [457, 699, 110, 113],
    'category_id': 0,
    'segmentation': [[494,
      730,
      505,
      717,
      513,
      705,
      522,
      703,
      524,
      708,
      533,
      705,
     

In [27]:
metadata = MetadataCatalog.get("car_data")
metadata

namespace(name='car_data',
          json_file='./inference/annotations.json',
          image_root='./inference/test_img',
          evaluator_type='coco',
          thing_classes=['dent', 'scratch', 'emblem', 'lamp'],
          thing_dataset_id_to_contiguous_id={1: 0, 2: 1, 3: 2, 4: 3})

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # only has one class (damage) + 1
cfg.MODEL.RETINANET.NUM_CLASSES = 4 # only has one class (damage) + 1
#cfg.MODEL.WEIGHTS = os.path.join("/home/adminuser/notebooks/01seokhwa/models/model_final.pth")
cfg.MODEL.WEIGHTS = os.path.join('./inference/model/model_final.pth')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 
cfg['MODEL']['DEVICE']='cuda'#or cpu
predictor = DefaultPredictor(cfg)

damage_class_map= {0:'dent', 1:'scratch'}

dataset = DatasetCatalog.get("car_data")
metadata = MetadataCatalog.get("car_data")


fig, (ax1) = plt.subplots(1, figsize =(16,12))


img = io.imread('./inference/test_img/5.jpg')
model_output = predictor(img)
model_output = model_output["instances"].to("cpu")
# only include detected instances with high enough score
ni = model_output[model_output.pred_classes != 3]
ni = ni[ni.pred_classes != 2]
ni = ni[ni.scores > 0.8]

# use the built-in visualizer to draw the boxes onto the image
v = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get("car_data"))
img = v.draw_instance_predictions(model_output).get_image()
plt.imshow(img)
plt.show()

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (6, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (6,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (20, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (20,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (5, 256, 1, 1) in the checkpoint but (4, 256, 1, 1) in th

[08/25 01:32:22 d2.data.datasets.coco]: Loaded 461 images in COCO format from ./inference/annotations.json
